In [ ]:
import pandas as pd
import polars as pl
import rtsvg
rt = rtsvg.RACETrack()
import os
import kagglehub
path = kagglehub.dataset_download("new-york-city/nyc-citywide-payroll-data")
os.listdir(path)
df = pl.read_csv(path+"/Citywide_Payroll_Data__Fiscal_Year_.csv")
# Remove spaces in first (or last) slot
for _column_ in ['Pay Basis', 'Work Location Borough']:
    df = df.with_columns(pl.col(_column_).str.strip_chars(' ').alias(_column_))
#df = rt.columnsAreTimestamps(df, 'Agency Start Date')
# To Upper Case
for _column_ in ['Agency Name', 'Last Name', 'First Name', 'Mid Init', 'Work Location Borough', 'Title Description']:
    df = df.with_columns(pl.col(_column_).str.to_uppercase().alias(_column_))
# Remove "$" sign... cast to float
for _column_ in ['Base Salary', 'Regular Gross Paid', 'Total OT Paid', 'Total Other Pay']:
    df = df.with_columns(pl.col(_column_).str.strip_chars_start('$').cast(pl.Float64).alias(_column_))
print(f'{len(df):_}')
print(df.columns)

In [ ]:
_parms_ = {'color_by':'Pay Basis'}
_tiles_ = []
_list_  = list(df.columns)
for i in range(len(_list_)): _tiles_.append(rt.histogram(df, bin_by=_list_[i], **_parms_))
rt.tile(_tiles_)

In [ ]:
#_len_ = 40
#for x in sorted(list(set(df['Agency Name']))):
#    _other_ = f'"{x}"'
#    _str_   = f'"{x.strip()}"'
#    print(f'{_other_:{_len_}}:({_str_:{_len_}},{_str_}),')

In [ ]:
_fix_ = {
"ADMIN FOR CHILDREN'S SVCS     "        :("ADMIN FOR CHILDREN'S SVCS"             ,"None"),
"ADMIN TRIALS AND HEARINGS     "        :("ADMIN TRIALS AND HEARINGS"             ,"None"),
"BOARD OF CORRECTION           "        :("BOARD OF CORRECTIONS"                  ,"None"),
"BOARD OF CORRECTIONS          "        :("BOARD OF CORRECTIONS"                  ,"None"),
"BOARD OF ELECTION             "        :("BOARD OF ELECTIONS"                    ,"None"),
"BOARD OF ELECTION POLL WORKERS"        :("BOARD OF ELECTIONS"                    ,"POLL WORKERS"),
"BOROUGH PRESIDENT-BRONX       "        :("BOROUGH PRESIDENT"                     ,"BRONX"),
"BOROUGH PRESIDENT-BROOKLYN    "        :("BOROUGH PRESIDENT"                     ,"BROOKLYN"),
"BOROUGH PRESIDENT-QUEENS      "        :("BOROUGH PRESIDENT"                     ,"QUEENS"),
"BOROUGH PRESIDENT-STATEN IS   "        :("BOROUGH PRESIDENT"                     ,"STATEN ISLAND"),
"BRONX COMMUNITY BOARD #1      "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #1"),
"BRONX COMMUNITY BOARD #10     "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #10"),
"BRONX COMMUNITY BOARD #11     "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #11"),
"BRONX COMMUNITY BOARD #12     "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #12"),
"BRONX COMMUNITY BOARD #2      "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #2"),
"BRONX COMMUNITY BOARD #3      "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #3"),
"BRONX COMMUNITY BOARD #4      "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #4"),
"BRONX COMMUNITY BOARD #5      "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #5"),
"BRONX COMMUNITY BOARD #6      "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #6"),
"BRONX COMMUNITY BOARD #7      "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #7"),
"BRONX COMMUNITY BOARD #8      "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #8"),
"BRONX COMMUNITY BOARD #9      "        :("BRONX COMMUNITY BOARD"                 ,"BOARD #9"),
"BRONX DA       "                       :("DISTRICT ATTORNEY"                     ,"BRONX"),
"BRONX DISTRICT ATTORNEY       "        :("DISTRICT ATTORNEY"                     ,"BRONX"),
"BROOKLYN COMMUNITY BOARD #1   "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #1"),
"BROOKLYN COMMUNITY BOARD #10  "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #10"),
"BROOKLYN COMMUNITY BOARD #11  "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #11"),
"BROOKLYN COMMUNITY BOARD #12  "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #12"),
"BROOKLYN COMMUNITY BOARD #13  "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #13"),
"BROOKLYN COMMUNITY BOARD #14  "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #14"),
"BROOKLYN COMMUNITY BOARD #15  "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #15"),
"BROOKLYN COMMUNITY BOARD #16  "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #16"),
"BROOKLYN COMMUNITY BOARD #17  "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #17"),
"BROOKLYN COMMUNITY BOARD #18  "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #18"),
"BROOKLYN COMMUNITY BOARD #2   "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #2"),
"BROOKLYN COMMUNITY BOARD #3   "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #3"),
"BROOKLYN COMMUNITY BOARD #4   "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #4"),
"BROOKLYN COMMUNITY BOARD #5   "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #5"),
"BROOKLYN COMMUNITY BOARD #6   "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #6"),
"BROOKLYN COMMUNITY BOARD #7   "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #7"),
"BROOKLYN COMMUNITY BOARD #8   "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #8"),
"BROOKLYN COMMUNITY BOARD #9   "        :("BROOKLYN COMMUNITY BOARD"              ,"BOARD #9"),
"BUSINESS INTEGRITY COMMISSION "        :("BUSINESS INTEGRITY COMMISSION"         ,"None"),
"CAMPAIGN FINANCE BOARD        "        :("CAMPAIGN FINANCE BOARD"                ,"None"),
"CITY CLERK                    "        :("CITY CLERK"                            ,"None"),
"CITY COUNCIL                  "        :("CITY COUNCIL"                          ,"None"),
"CIVIL SERVICE COMMISSION      "        :("CIVIL SERVICE COMMISSION"              ,"None"),
"CIVILIAN COMPLAINT REVIEW BD  "        :("CIVILIAN COMPLAINT REVIEW BOARD"       ,"None"),
"COMMUNITY COLLEGE (BRONX)     "        :("COMMUNITY COLLEGE"                     ,"BRONX"),
"COMMUNITY COLLEGE (HOSTOS)    "        :("COMMUNITY COLLEGE"                     ,"HOSTOS"),
"COMMUNITY COLLEGE (KINGSBORO) "        :("COMMUNITY COLLEGE"                     ,"KINGSBORO"),
"COMMUNITY COLLEGE (LAGUARDIA) "        :("COMMUNITY COLLEGE"                     ,"LAGUARDIA"),
"COMMUNITY COLLEGE (MANHATTAN) "        :("COMMUNITY COLLEGE"                     ,"MANHATTAN"),
"COMMUNITY COLLEGE (QUEENSBORO)"        :("COMMUNITY COLLEGE"                     ,"QUEENSBORO"),
"CONFLICTS OF INTEREST BOARD   "        :("CONFLICTS OF INTEREST BOARD"           ,"None"),
"CONSUMER AFFAIRS              "        :("CONSUMER AFFAIRS"                      ,"None"),
"CULTURAL AFFAIRS              "        :("CULTURAL AFFAIRS"                      ,"None"),
"CUNY CENTRAL OFFICE           "        :("CUNY CENTRAL OFFICE"                   ,"None"),
"DEPARTMENT FOR THE AGING      "        :("DEPARTMENT FOR THE AGING"              ,"None"),
"DEPARTMENT OF BUILDINGS       "        :("DEPARTMENT OF BUILDINGS"               ,"None"),
"DEPARTMENT OF BUSINESS SERV.  "        :("DEPARTMENT OF BUSINESS SERVICES"       ,"None"),
"DEPARTMENT OF CITY PLANNING   "        :("DEPARTMENT OF CITY PLANNING"           ,"None"),
"DEPARTMENT OF CORRECTION      "        :("DEPARTMENT OF CORRECTION"              ,"None"),
"DEPARTMENT OF EDUCATION ADMIN "        :("DEPARTMENT OF EDUCATION"               ,"Admin"),
"DEPARTMENT OF FINANCE         "        :("DEPARTMENT OF FINANCE"                 ,"None"),
"DEPARTMENT OF INVESTIGATION   "        :("DEPARTMENT OF INVESTIGATION"           ,"None"),
"DEPARTMENT OF JUVENILE JUSTICE"        :("DEPARTMENT OF JUVENILE JUSTICE"        ,"None"),
"DEPARTMENT OF PROBATION       "        :("DEPARTMENT OF PROBATION"               ,"None"),
"DEPARTMENT OF SANITATION      "        :("DEPARTMENT OF SANITATION"              ,"None"),
"DEPARTMENT OF TRANSPORTATION  "        :("DEPARTMENT OF TRANSPORTATION"          ,"None"),
"DEPT OF CITYWIDE ADMIN SVCS   "        :("DEPT OF CITYWIDE ADMIN SERVICES"       ,"None"),
"DEPT OF ED HRLY SUPPORT STAFF "        :("DEPT OF EDUCATION"                     ,"HOURLY SUPPORT STAFF"),
"DEPT OF ED PARA PROFESSIONALS "        :("DEPT OF EDUCATION"                     ,"PARA PROFESSIONALS"),
"DEPT OF ED PEDAGOGICAL        "        :("DEPT OF EDUCATION"                     ,"PEDAGOGICAL"),
"DEPT OF ED PER DIEM TEACHERS  "        :("DEPT OF EDUCATION"                     ,"PER DIEM TEACHERS"),
"DEPT OF ED PER SESSION TEACHER"        :("DEPT OF EDUCATION"                     ,"PER SESSION TEACHER"),
"DEPT OF ENVIRONMENT PROTECTION"        :("DEPT OF ENVIRONMENT PROTECTION"        ,"None"),
"DEPT OF HEALTH/MENTAL HYGIENE "        :("DEPT OF HEALTH/MENTAL HYGIENE"         ,"None"),
"DEPT OF INFO TECH & TELECOMM  "        :("DEPT OF INFO TECH & TELECOMM"          ,"None"),
"DEPT OF PARKS & RECREATION    "        :("DEPT OF PARKS & RECREATION"            ,"None"),
"DEPT OF RECORDS & INFO SERVICE"        :("DEPT OF RECORDS & INFO SERVICE"        ,"None"),
"DEPT OF YOUTH & COMM DEV SRVS "        :("DEPT OF YOUTH & COMM DEV SRVS"         ,"None"),
"DEPT. OF DESIGN & CONSTRUCTION"        :("DEPT OF DESIGN & CONSTRUCTION"         ,"None"),
"DEPT. OF HOMELESS SERVICES    "        :("DEPT OF HOMELESS SERVICES"             ,"None"),
"DISTRICT ATTORNEY KINGS COUNTY"        :("DISTRICT ATTORNEY"                     ,"BROOKLYN"), # apparently, kings county is actually brooklyn
"DISTRICT ATTORNEY QNS COUNTY  "        :("DISTRICT ATTORNEY"                     ,"QNS COUNTY"),
"DISTRICT ATTORNEY RICHMOND COU"        :("DISTRICT ATTORNEY"                     ,"RICHMOND COUNTY"),
"DISTRICT ATTORNEY-MANHATTAN   "        :("DISTRICT ATTORNEY"                     ,"MANHATTAN"),
"DISTRICT ATTORNEY-SPECIAL NARC"        :("DISTRICT ATTORNEY"                     ,"SPECIAL NARC"),
"DISTRICTING COMMISSION        "        :("DISTRICTING COMMISSION"                ,"None"),
"DOE CUSTODIAL PAYROL          "        :("DOE CUSTODIAL PAYROL"                  ,"None"),
"EQUAL EMPLOY PRACTICES COMM   "        :("EQUAL EMPLOY PRACTICES COMMISSION"     ,"None"),
"FINANCIAL INFO SVCS AGENCY    "        :("FINANCIAL INFO SERVICES AGENCY"        ,"None"),
"FIRE DEPARTMENT               "        :("FIRE DEPARTMENT"                       ,"None"),
"GUTTMAN COMMUNITY COLLEGE     "        :("GUTTMAN COMMUNITY COLLEGE"             ,"None"),
"HOUSING PRESERVATION & DVLPMNT"        :("HOUSING PRESERVATION & DEVELOPMENT"    ,"None"),
"HRA/DEPT OF SOCIAL SERVICES   "        :("DEPT OF SOCIAL SERVICES"               ,"HRA"),
"HUMAN RIGHTS COMMISSION       "        :("HUMAN RIGHTS COMMISSION"               ,"None"),
"HUNTER COLLEGE HIGH SCHOOL    "        :("HUNTER COLLEGE HIGH SCHOOL"            ,"None"),
"INDEPENDENT BUDGET OFFICE     "        :("INDEPENDENT BUDGET OFFICE"             ,"None"),
"KINGS DA"                              :("DISTRICT ATTORNEY"                     ,"BROOKLYN"), # apparently, kings county is actually brooklyn
"LANDMARKS PRESERVATION COMM   "        :("LANDMARKS PRESERVATION COMM"           ,"None"),
"LAW DEPARTMENT                "        :("LAW DEPARTMENT"                        ,"None"),
"MANHATTAN COMMUNITY BOARD #1  "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #1"),
"MANHATTAN COMMUNITY BOARD #10 "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #10"),
"MANHATTAN COMMUNITY BOARD #11 "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #11"),
"MANHATTAN COMMUNITY BOARD #12 "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #12"),
"MANHATTAN COMMUNITY BOARD #2  "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #2"),
"MANHATTAN COMMUNITY BOARD #3  "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #3"),
"MANHATTAN COMMUNITY BOARD #4  "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #4"),
"MANHATTAN COMMUNITY BOARD #5  "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #5"),
"MANHATTAN COMMUNITY BOARD #6  "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #6"),
"MANHATTAN COMMUNITY BOARD #7  "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #7"),
"MANHATTAN COMMUNITY BOARD #8  "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #8"),
"MANHATTAN COMMUNITY BOARD #9  "        :("MANHATTAN COMMUNITY BOARD"             ,"BOARD #9"),
"MANHATTAN DA   "                       :("DISTRICT ATTORNEY"                     ,"MANHATTAN"),
"MAYORS OFFICE OF CONTRACT SVCS"        :("MAYORS OFFICE OF CONTRACT SERVICES"    ,"None"),
"MUNICIPAL WATER FIN AUTHORITY "        :("MUNICIPAL WATER FINANCIAL AUTHORITY"   ,"None"),
"NYC DEPT OF VETERANS' SERVICES"        :("DEPT OF VETERANS' SERVICES"            ,"None"),
"NYC EMPLOYEES RETIREMENT SYS  "        :("EMPLOYEES RETIREMENT SYSTEM"           ,"None"),
"NYC HOUSING AUTHORITY         "        :("HOUSING AUTHORITY"                     ,"None"),
"NYC POLICE PENSION FUND       "        :("POLICE DEPARTMENT}"                    ,"PENSION FUND"),
"OFF OF PAYROLL ADMINISTRATION "        :("OFFICE OF PAYROLL ADMINISTRATION"      ,"None"),
"OFFICE OF COLLECTIVE BARGAININ"        :("OFFICE OF COLLECTIVE BARGAININ"        ,"None"),
"OFFICE OF EMERGENCY MANAGEMENT"        :("OFFICE OF EMERGENCY MANAGEMENT"        ,"None"),
"OFFICE OF LABOR RELATIONS     "        :("OFFICE OF LABOR RELATIONS"             ,"None"),
"OFFICE OF MANAGEMENT & BUDGET "        :("OFFICE OF MANAGEMENT & BUDGET"         ,"None"),
"OFFICE OF THE ACTUARY         "        :("OFFICE OF THE ACTUARY"                 ,"None"),
"OFFICE OF THE COMPTROLLER     "        :("OFFICE OF THE COMPTROLLER"             ,"None"),
"OFFICE OF THE MAYOR           "        :("OFFICE OF THE MAYOR"                   ,"None"),
"PERSONNEL MONITORS            "        :("PERSONNEL MONITORS"                    ,"None"),
"POLICE DEPARTMENT"                     :("POLICE DEPARTMENT"                     ,"None"),
"POLICE DEPARTMENT             "        :("POLICE DEPARTMENT"                     ,"None"),
"PRESIDENT BOROUGH OF MANHATTAN"        :("PRESIDENT BOROUGH"                     ,"MANHATTAN"),
"PUBLIC ADMINISTRATOR-BRONX    "        :("PUBLIC ADMINISTRATOR"                  ,"BRONX"),
"PUBLIC ADMINISTRATOR-KINGS    "        :("PUBLIC ADMINISTRATOR"                  ,"BROOKLYN"),
"PUBLIC ADMINISTRATOR-NEW YORK "        :("PUBLIC ADMINISTRATOR"                  ,"NEW YORK"),
"PUBLIC ADMINISTRATOR-QUEENS   "        :("PUBLIC ADMINISTRATOR"                  ,"QUEENS"),
"PUBLIC ADMINISTRATOR-RICHMOND "        :("PUBLIC ADMINISTRATOR"                  ,"RICHMOND"),
"PUBLIC ADVOCATE               "        :("PUBLIC ADVOCATE"                       ,"None"),
"PUBLIC SERVICE CORPS          "        :("PUBLIC SERVICE CORPS"                  ,"None"),
"QUEENS COMMUNITY BOARD #1     "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #1"),
"QUEENS COMMUNITY BOARD #10    "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #10"),
"QUEENS COMMUNITY BOARD #11    "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #11"),
"QUEENS COMMUNITY BOARD #12    "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #12"),
"QUEENS COMMUNITY BOARD #13    "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #13"),
"QUEENS COMMUNITY BOARD #14    "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #14"),
"QUEENS COMMUNITY BOARD #2     "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #2"),
"QUEENS COMMUNITY BOARD #3     "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #3"),
"QUEENS COMMUNITY BOARD #4     "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #4"),
"QUEENS COMMUNITY BOARD #5     "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #5"),
"QUEENS COMMUNITY BOARD #6     "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #6"),
"QUEENS COMMUNITY BOARD #7     "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #7"),
"QUEENS COMMUNITY BOARD #8     "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #8"),
"QUEENS COMMUNITY BOARD #9     "        :("QUEENS COMMUNITY BOARD"                ,"BOARD #9"),
"QUEENS DA  "                           :("DISTRICT ATTORNEY"                     ,"QUEENS"),
"RICHMOND DA"                           :("DISTRICT ATTORNEY"                     ,"RICHMOND"),
"SPEC NARCS-DA"                         :("DISTRICT ATTORNEY"                     ,"SPEC NARCS"),
"STATEN ISLAND COMMUNITY BD #1 "        :("STATEN ISLAND COMMUNITY BOARD"         ,"BOARD #1"),
"STATEN ISLAND COMMUNITY BD #2 "        :("STATEN ISLAND COMMUNITY BOARD"         ,"BOARD #2"),
"STATEN ISLAND COMMUNITY BD #3 "        :("STATEN ISLAND COMMUNITY BOARD"         ,"BOARD #3"),
"TAX COMMISSION                "        :("TAX COMMISSION"                        ,"None"),
"TAXI & LIMOUSINE COMMISSION   "        :("TAXI & LIMOUSINE COMMISSION"           ,"None"),
"TEACHERS RETIREMENT SYSTEM    "        :("DEPT OF EDUCATION"                     ,"TEACHERS RETIREMENT SYSTEM"),
}